In [1]:
#no own code here, following instructional video
import pandas as pd
from ydata_profiling import ProfileReport

In [ ]:
df = pd.read_csv("cars.csv")
df = df.drop(columns = 'index')

In [112]:
profile = ProfileReport(df, title = 'Profile report')
profile.to_file('./card-profile.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/home/tuukka/data_engineering/venv/lib/python3.12/site-packages/ydata_profiling/model/describe.py:85: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_start = datetime.utcnow()
Summarize dataset: 100%|██████████| 27/27 [00:01<00:00, 21.62it/s, Completed]                         /home/tuukka/data_engineering/venv/lib/python3.12/site-packages/ydata_profiling/model/describe.py:190: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_end = datetime.utcnow()
Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]/home/tuukka/data_engineering/venv/lib/python3.12/site-packages/matplotlib/cbook.py:684: DeprecationWar

In [113]:
#profile

In [114]:
#Great expectations
import great_expectations as gx

In [115]:
context = gx.get_context()
assert type(context).__name__ == "EphemeralDataContext"

In [116]:
data_source = context.data_sources.add_pandas(name="cars")
data_asset = data_source.add_dataframe_asset(name="cars_asset")

In [117]:
batch_definition_name = 'cars_batch'
batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)
assert batch_definition_name == batch_definition_name

In [118]:
df = pd.read_csv("cars.csv")
batch_parameters = {"dataframe": df}
batch = batch_definition.get_batch(batch_parameters = batch_parameters)


In [119]:
batch_parameters = {'dataframe':df}
batch = batch_definition.get_batch(batch_parameters=batch_parameters)

In [120]:
expectation_suite_name ="cars_suite"
suite = gx.ExpectationSuite(name=expectation_suite_name)

suite.add_expectation(
    gx.expectations.ExpectColumnDistinctValuesToBeInSet(
        column="Transmission",
        value_set= ['1','2','3','4','5','7']
    )
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeUnique(column="index")
)
context.suites.add(suite)


{
  "name": "cars_suite",
  "id": "3ce5d534-51d1-4a28-aa80-fa80a105b389",
  "expectations": [
    {
      "type": "expect_column_distinct_values_to_be_in_set",
      "kwargs": {
        "column": "Transmission",
        "value_set": [
          "1",
          "2",
          "3",
          "4",
          "5",
          "7"
        ]
      },
      "meta": {},
      "id": "3ea8c062-ca2e-4380-a3d8-2aca04d42d3a",
      "severity": "critical"
    },
    {
      "type": "expect_column_values_to_be_unique",
      "kwargs": {
        "column": "index"
      },
      "meta": {},
      "id": "6eaf20de-9e38-4f48-bb0d-bf6b95056dae",
      "severity": "critical"
    }
  ],
  "meta": {
    "great_expectations_version": "1.9.3"
  },
  "notes": null
}

In [121]:
validation_results = batch.validate(suite)

if "meta" in validation_results and "run_id" not in validation_results['meta']:
    validation_results['meta']['run_id'] = 'manual_run'


from great_expectations.render.renderer import ValidationResultsPageRenderer
from great_expectations.render.view import DefaultJinjaPageView

renderer = ValidationResultsPageRenderer()

rendered = renderer.render(validation_results)
html = DefaultJinjaPageView().render(rendered)





Calculating Metrics: 100%|██████████| 11/11 [00:00<00:00, 741.48it/s]
An unexpected Exception occurred during data docs rendering.  Because of this error, certain parts of data docs will not be rendered properly and/or may not appear altogether.  Please use the trace, included in this message, to diagnose and repair the underlying issue.  Detailed information follows:
            TypeError: "num_to_str received an invalid value: 0.0 of type float64.".  Traceback: "Traceback (most recent call last):
  File "/home/tuukka/data_engineering/venv/lib/python3.12/site-packages/great_expectations/render/util.py", line 64, in num_to_str
    d = local_context.create_decimal(s)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
decimal.InvalidOperation: [<class 'decimal.ConversionSyntax'>]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tuukka/data_engineering/venv/lib/python3.12/site-packages/great_expectations/render/renderer/content_bl

In [122]:
with open("cars_validation_report.html", "w", encoding = "utf-8") as f:
    f.write(html)
print("complete")

complete
